In [25]:
!pip install -q -U google-generativeai

In [36]:
from google.colab import userdata
#print(userdata.get('Gemini_API'))


In [58]:
import google.generativeai as genai
import pandas as pd, json, re
import time
import os

In [39]:

# os.environ['Gemini_API'] = userdata.get('Gemini_API')
# print(os.getenv('Gemini_API'))

In [38]:
genai.configure(api_key=userdata.get('Gemini_API'))

**TO generate active to passive**

In [55]:

model = genai.GenerativeModel("models/gemini-2.5-flash")

n = 50
# prompt = f"""
# Generate {n} unique English sentence pairs converting Active Voice to Passive Voice.

# Return strictly as JSON list:
# [
#   {{"original": "The chef cooks the meal.", "transformed": "The meal is cooked by the chef."}},
#   ...
# ]

# Rules:
# - Use grammatically correct and natural English sentences.
# - Do NOT include explanations, numbering, or markdown.
# - Only return the JSON list.
# """

# response = model.generate_content(prompt)
# raw_output = response.text
# print("Raw output preview:\n", raw_output[:400])

In [56]:
transformations = [
    {
        "label": "Active to Passive",
        "instruction": "Convert each Active Voice sentence to Passive Voice."
    },
    {
        "label": "Passive to Active",
        "instruction": "Convert each Passive Voice sentence to Active Voice."
    },
    {
        "label": "Direct to Indirect Speech",
        "instruction": "Convert each Direct Speech sentence to Indirect Speech."
    },
    {
        "label": "Indirect to Direct Speech",
        "instruction": "Convert each Indirect Speech sentence to Direct Speech."
    },
    {
        "label": "Positive to Negative",
        "instruction": "Convert each Positive statement to its Negative version."
    },
    {
        "label": "Negative to Positive",
        "instruction": "Convert each Negative statement to its Positive version."
    }
]

In [61]:
all_dfs = []
for t in transformations:
    print(f"\n Generating: {t['label']} ...")
    prompt = f"""
Generate {n} unique English sentence pairs.
Task: {t['instruction']}

Return strictly as JSON list:
[
  {{"original": "Example original sentence", "transformed": "Example transformed version"}},
  ...
]

Rules:
- Use grammatically correct, natural English sentences.
- No numbering, markdown, or explanations.
- Output only a valid JSON list.
"""

    #  Generate text using Gemini
    response = model.generate_content(prompt)
    raw_output = response.text

    #  Extract JSON safely
    try:
        json_text = re.search(r"\[.*\]", raw_output, re.DOTALL).group(0)
        pairs = json.loads(json_text)
        df = pd.DataFrame(pairs)
        df["Label"] = t["label"]
        df.to_csv(f"/content/{t['label'].replace(' ', '_')}.csv", index=False)
        all_dfs.append(df)
        print(f" {t['label']} → {len(df)} pairs saved.")
    except Exception as e:
        print(f" Failed for {t['label']}: {e}")
        print(raw_output[:400])

    time.sleep(2)


 Generating: Active to Passive ...
 Active to Passive → 50 pairs saved.

 Generating: Passive to Active ...
 Passive to Active → 50 pairs saved.

 Generating: Direct to Indirect Speech ...
 Direct to Indirect Speech → 50 pairs saved.

 Generating: Indirect to Direct Speech ...
 Indirect to Direct Speech → 50 pairs saved.

 Generating: Positive to Negative ...
 Positive to Negative → 50 pairs saved.

 Generating: Negative to Positive ...
 Negative to Positive → 50 pairs saved.


In [63]:
if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=True).drop_duplicates()
    final_df.to_csv("/content/ATI_GENERATED_grammar_transformation_dataset.csv", index=False)
    print("\n All transformations completed.")
    print("Final dataset saved at /content/ATI_GENERATED_grammar_transformation_dataset.csv")
    print(final_df["Label"].value_counts())
    display(final_df.head(10))
else:
    print("No valid data generated — check your API key or responses.")


 All transformations completed.
Final dataset saved at /content/grammar_transformation_dataset.csv
Label
Active to Passive            50
Passive to Active            50
Direct to Indirect Speech    50
Indirect to Direct Speech    50
Positive to Negative         50
Negative to Positive         50
Name: count, dtype: int64


,original,transformed,Label
0,The chef bakes delicious cakes.,Delicious cakes are baked by the chef.,Active to Passive
1,She is writing a new novel.,A new novel is being written by her.,Active to Passive
2,They have completed the project.,The project has been completed by them.,Active to Passive
3,My sister painted that picture.,That picture was painted by my sister.,Active to Passive
4,The students were discussing the topic.,The topic was being discussed by the students.,Active to Passive
5,He had finished his homework before dinner.,His homework had been finished by him before d...,Active to Passive
6,The company will launch a new product next month.,A new product will be launched by the company ...,Active to Passive
7,You can solve this puzzle easily.,This puzzle can be solved easily by you.,Active to Passive
8,We should protect endangered species.,Endangered species should be protected by us.,Active to Passive
9,The strong winds damaged the old tree.,The old tree was damaged by the strong winds.,Active to Passive
